Mount Google Drive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
train_dir = '/content/drive/MyDrive/disease_skin/train'
test_dir = '/content/drive/MyDrive/disease_skin/test'

Augmentasi Data

In [29]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [30]:
# Image size
img_size = (224, 224)

# Training with more augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [31]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 320 images belonging to 5 classes.
Found 80 images belonging to 5 classes.


In [33]:
# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

Found 100 images belonging to 5 classes.


In [34]:
# Load InceptionV3 with pretrained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze the base model (more layers)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [35]:
# Compile the model
opt = Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [36]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[LearningRateScheduler(lr_scheduler), early_stopping]
)

Epoch 1/50
10/10 [==============================] - 41s 1s/step - loss: 1.4360 - accuracy: 0.3750 - val_loss: 1.1957 - val_accuracy: 0.4250 - lr: 1.0000e-04
Epoch 2/50
10/10 [==============================] - 9s 915ms/step - loss: 0.7235 - accuracy: 0.7750 - val_loss: 1.4880 - val_accuracy: 0.4375 - lr: 1.0000e-04
Epoch 3/50
10/10 [==============================] - 8s 800ms/step - loss: 0.4845 - accuracy: 0.8438 - val_loss: 1.6842 - val_accuracy: 0.4625 - lr: 1.0000e-04
Epoch 4/50
10/10 [==============================] - 10s 973ms/step - loss: 0.3126 - accuracy: 0.8844 - val_loss: 1.5369 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 5/50
10/10 [==============================] - 10s 988ms/step - loss: 0.1978 - accuracy: 0.9406 - val_loss: 1.1769 - val_accuracy: 0.6625 - lr: 1.0000e-04
Epoch 6/50
10/10 [==============================] - 8s 777ms/step - loss: 0.1905 - accuracy: 0.9531 - val_loss: 1.4079 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 7/50
10/10 [============================

In [37]:
# Evaluation
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

4/4 [==============================] - 1s 193ms/step - loss: 0.4279 - accuracy: 0.8600
Test Accuracy: 86.00%


In [38]:
# Classification Report
from sklearn.metrics import classification_report
import numpy as np

# Predict classes for test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# True classes for test set
true_classes = test_generator.classes

# Class labels
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

4/4 [==============================] - 2s 213ms/step
              precision    recall  f1-score   support

  Basal Cell       0.24      0.25      0.24        20
  Dermatitis       0.24      0.35      0.29        20
      Eczema       0.10      0.05      0.07        20
 Melanocytic       0.28      0.25      0.26        20
    Melanoma       0.27      0.30      0.29        20

    accuracy                           0.24       100
   macro avg       0.23      0.24      0.23       100
weighted avg       0.23      0.24      0.23       100

